# Amazon reviews: Kindle Store Category

# Instructions ⚠
- Do not use "Run All" because the code logic varies in different sections of this article.
- Running the code in each Cell in sequential order is OK.

## Observation data

Use the pandas library to read the data set (kindle_review.csv).

In [16]:
import pandas as pd
data = pd.read_csv('./kindle_reviews.csv')

View data set dimensions (number of rows, number of columns)

In [17]:
data.shape

(982619, 10)

View column name

In [18]:
data.columns

Index(['Unnamed: 0', 'asin', 'helpful', 'overall', 'reviewText', 'reviewTime',
       'reviewerID', 'reviewerName', 'summary', 'unixReviewTime'],
      dtype='object')

"Unnamed: 0" is the serial number automatically generated after the original data is processed by pandas, which can be deleted.
> axis: {0 or ‘index’, 1 or ‘columns’}, default 0
> Whether to drop labels from the index (0 or ‘index’) or columns (1 or ‘columns’).

In [19]:
data = data.drop('Unnamed: 0',axis='columns')

Check to see if the first column has been deleted

In [20]:
data.columns

Index(['asin', 'helpful', 'overall', 'reviewText', 'reviewTime', 'reviewerID',
       'reviewerName', 'summary', 'unixReviewTime'],
      dtype='object')

Some data is not fixed length, you need to view the maximum length of various data

In [52]:
# Take each column, measure the length of all the rows below it, store it in the list, and find the maximum length
for col in ['asin', 'reviewText', 'reviewerID','reviewerName', 'summary']:
    # Some of the data contains numbers that need to be turned into the string str
    max_length = max([len(str(i)) for i in data[col]])
    print(f'max length of {col} is {max_length}')


max length of asin is 10
max length of reviewText is 23037
max length of reviewerID is 21
max length of reviewerName is 49
max length of summary is 325


Some columns may have empty (NaN) data in them and need to be marked and set to before inserting into the database

In [153]:
cols = ['asin', 'helpful', 'overall','reviewText','reviewTime','reviewerID','reviewerName','summary','unixReviewTime']
for col in cols:
    if data[col].isnull().values.any():    
        print(col)

reviewText
reviewerName
summary


## Entity Relationship Analysis

This data set of entities have **Customer** and **Book**, There is a **buying relationship** between the customer and the book, **The purchase relationship** is derived from the evaluation.

The attributes of a Customer are:
| | attribute column name | format
|--|--|--|
| | ID reviewerID | | string length for the 20 or so
| | | name reviewerName | strings

The properties of a Book are:
| | | attribute column name | format
|--|--|--|
| | ID asin (Amazon Standard Identification Number) | length of ten strings

The attributes of Review are:
| | | attribute column name | format
|--|--|--|
| | useful degree helpful [digital, digital] | | list
| | | score overall | Numbers
Evaluation words | | reviewText | string, longer |
| | | evaluation date reviewTime | strings
Short summary | | evaluation summary | string, |
Evaluation timestamp | | unixReviewTime for UNIX timestamp | | format

## create database
Use SQLAlchemy to manipulate the database.

Test database connection, query all databases on MySQL.

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy import text
engine = create_engine('mysql+pymysql://用户名:密码@IP地址:端口号/数据库')
with engine.connect() as conn:
    query = 'show databases'
    print(conn.execute(text(query)).fetchall())

### Definition model

In [3]:
from sqlalchemy import Column
from sqlalchemy import create_engine
from sqlalchemy import String, Text, Date, DateTime, Integer
from sqlalchemy import ForeignKey
from sqlalchemy.orm import declarative_base
from sqlalchemy.orm import Mapped
from sqlalchemy import text
from datetime import datetime


Base = declarative_base()

# max length of asin is 10
# max length of reviewText is 23037
# max length of reviewerID is 21
# max length of reviewerName is 49
# max length of summary is 325


class Customer(Base):
    __tablename__ = 'customer'
    id = Column(String(30), primary_key=True)
    name = Column(String(100))

    def __repr__(self) -> str:
        return f'id:{self.id},name:{self.name}'


class Book(Base):
    __tablename__ = 'book'
    id = Column(String(10), primary_key=True)

    def __repr__(self) -> str:
        return f'id:{self.id}'


class Review(Base):
    __tablename__ = 'review'
    customer_id = Column(ForeignKey('customer.id'), primary_key=True)
    book_id = Column(ForeignKey('book.id'), primary_key=True)
    review_text = Column(Text)
    review_time = Column(Date)
    helpful = Column(Integer)
    not_helpful = Column(Integer)
    summary = Column(Text)
    unix_time_stamp = Column(DateTime)

    def __repr__(self) -> str:
        return f'customer_id:{self.customer_id}, book_id:{self.book_id}, text:{self.review_text}, date:{self.review_time}, timestamp:{self.unix_time_stamp}'

   

In [ ]:
# create database
engine = create_engine(
    'mysql+pymysql://用户名:密码@IP地址:端口号/数据库')
my_db = 'AmazonBookReview'
with engine.connect() as conn:
    query = f'CREATE DATABASE {my_db}'
    conn.execute(text(query))

In [ ]:
# create table
engine = create_engine(
    f'mysql+pymysql://用户名:密码@IP地址:端口号/{my_db}')
Base.metadata.create_all(engine)

In [ ]:
# check if create suceess
with engine.connect() as conn:
    query = f'SHOW TABLES;'
    print(conn.execute(text(query)).fetchall())

## Data reorganization
Extract data from CSV and use pandas to insert it into the database.

In [2]:
import pandas as pd
data = pd.read_csv('./kindle_reviews.csv')

### Customer
Define a dictionary to first store read data into a dictionary customer_dict (ID: name).
Considering that the same customer may appear multiple times in the data set, it is necessary to check whether the ID exists.
If the reviewerName column in the dataset is empty, check it with the isnull method and assign the value "if it is empty.

In [150]:
customer_dict = dict()
for index, row in data[['reviewerID', 'reviewerName']].iterrows():
    if row['reviewerID'] not in customer_dict:
        customer_dict[row['reviewerID']] = None if pd.isnull(
            row['reviewerName']) else row['reviewerName']


In [13]:
# Comparing the number of customers in customer_dict to the number of rows in the data is a big difference, and there is indeed a case of multiple user reviews
print('amount of customer =',len(customer_dict.keys()))
print('rows of data set = ',data.shape[0])

amount of customer = 68223
rows of data set =  982619


,Unnamed: 0,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime


Insert data

In [5]:
from sqlalchemy import engine,create_engine
my_db = 'AmazonBookReview'
engine = create_engine(
    f'mysql+pymysql://用户名:密码@IP地址:端口号/{my_db}')


Iterate over the dictionary customer_dict, creating objects with customer_id and customer_name, and store the objects in the list customers.
Use the add_all method to insert the list as data into the database's table customer.

In [152]:
from sqlalchemy.orm import Session

customers = list()
for customer_id, customer_name in customer_dict.items():
    customers.append(Customer(id=str(customer_id),name=customer_name ))

with Session(engine) as session:
    session.add_all(customers)
    session.commit()


In [20]:
# check how many data inserted
from sqlalchemy import text
with engine.connect() as conn:
    query = 'SELECT COUNT(1) FROM customer'
    print(conn.execute(text(query)).fetchall())

[(68223,)]


### Books

Use the collection to store the ID of the book, and the collection is automatically deduplicated.

In [3]:
books = set()
for book_id in data['asin']:
    books.add(book_id)

In [155]:
from sqlalchemy.orm import Session

book_list = [Book(id=book_id) for book_id in books]
with Session(engine) as session:
    session.add_all(book_list)
    session.commit()


In [26]:
# check how many data inserted
from sqlalchemy import text
with engine.connect() as conn:
    query = 'SELECT COUNT(1) FROM book'
    print(conn.execute(text(query)).fetchall())

[(61934,)]


### Reviews

In [4]:
from datetime import datetime
reviews = list()
for index, row in data[['reviewerID', 'asin', 'reviewText', 'reviewTime', 'helpful', 'summary', 'unixReviewTime']].iterrows():
    help_not = [int(i) for i in row['helpful'].removeprefix(
        '[').removesuffix(']').split(',')]
    reviews.append(Review(
        customer_id=str(row['reviewerID']), book_id=str(row['asin']),
        review_text=None if pd.isnull(
            row['reviewText']) else row['reviewText'],
        helpful=help_not[0], not_helpful=help_not[1],
        summary=None if pd.isnull(row['summary']) else row['summary'],
        review_time=datetime.strptime(row['reviewTime'], '%m %d, %Y').date(),
        unix_time_stamp=datetime.fromtimestamp(row['unixReviewTime'])))

print(len(reviews))


982619


In [6]:
from sqlalchemy.orm import Session

with Session(engine) as session:
    session.add_all(reviews)
    session.commit()
